In [ ]:
#라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorboardX import SummaryWriter
from torchinfo import summary

In [ ]:
#학습에 필요한 하이퍼파라미터
batch_size = 100
learning_rate = 0.0001
epoch = 15

In [ ]:
#데이터셋, 데이터로더
mnist_train = datasets.MNIST(root='../Data/', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = datasets.MNIST(root='../Data/', train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=1, drop_last=True)
#mnist의 이미지 사이즈는 [1, 28, 28] [채널, (이미지 사이즈)] 채널: (1 -> 흑백, 3 -> RGB)

In [ ]:
#CNN 모델 (Sequantial을 이용)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            # [100, 1, 28, 28] -> [100, 16, 24, 24]
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5),
            nn.ReLU(),
            
            # [100, 16, 24, 24] -> [100, 32, 20, 20]
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),
            nn.ReLU(),
            
            # [100, 32, 20, 20] -> [100, 32, 10, 10]
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # [100, 32, 10, 10] -> [100, 64, 6, 6]
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
            nn.ReLU(),
            
            # [100, 64, 6, 6] -> [100, 64, 3, 3]
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layer = nn.Sequential(
            # [100, 64 * 3 * 3] -> [100, 100]
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            
            # [100, 100] -> [100, 10]
            nn.Linear(100, 10)
        )
        
    def forward(self, x):
        x = self.layer(x)
        x = x.reshape(batch_size, -1)
        x = self.fc_layer(x)
        return x

In [ ]:
# 사용할 장치, 모델, 손실함수, 옵티마이저 선언
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 모델 출력
summary(model, (100, 1, 28, 28))

In [ ]:

# 학습
step_loss_arr = []
epoch_loss_arr = []
#writer = SummaryWriter()
for i in range(epoch):
    train_loader = tqdm(train_loader)
    epoch_loss = 0
    
    for j, [image, label] in enumerate(train_loader):
        image = image.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        
        output = model.forward(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        train_loader.set_postfix({"Loss" : loss})
        #writer.add_scalar('Loss/train', loss, i * len(train_loader) + j)
        epoch_loss += loss.item()
        step_loss_arr.append(loss.item())
        
    epoch_loss_arr.append(epoch_loss/len(train_loader))
#writer.flush()
#writer.close()

In [ ]:
# 평가
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for image, label in test_loader:
        image = image.to(device)
        label = label.to(device)
        
        output = model.forward(image)
        
        _, output_index = torch.max(output, 1)
        
        total += label.size(0)
        
        correct += (output_index == label).sum().float()
        
print("Accuracy of Test Data: {}%".format(100 * correct / total))